# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Markdown

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from utils.j_utils import load_env_vars, print_json
from utils.ui_utils import load_json
from langchain_openai import ChatOpenAI
import langchain_groq
from langchain_groq import ChatGroq
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, FileReadTool, DirectoryReadTool, MDXSearchTool

from crewai_tools import BaseTool
secrets_path = '.streamlit/secrets.toml'
research_data_path = 'data/research_example.json'
customer_support_data_path = 'data/customer_support_example.json'
sales_data_path = 'data/sales_example.json'
event_data_path = 'data/event_example.json'
venue_json_path = "output/venue_details.json"
instructions_path = "data/instructions"
financial_data_path = 'data/financial_example.json'
fake_resume_data_path = 'data/fake_resume.md'
resume_data_path = 'data/resume_example.json'

In [ ]:

research_data = load_json(research_data_path)
customer_support_data = load_json(customer_support_data_path)
sales_data = load_json(sales_data_path)
events_data = load_json(event_data_path)
financial_data = load_json(financial_data_path)
resume_data = load_json(resume_data_path)

In [ ]:
#OPEN AI DATA
model = "gpt-3.5-turbo"
api_key = load_env_vars("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=api_key, model=model)

In [ ]:
#GROQ DATA
model = "llama3-70b-8192"
api_key = load_env_vars("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model_name=model)

In [ ]:
scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)
serper_api_key = load_env_vars("SERPER_API_KEY")
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"
    
sentiment_analysis_tool = SentimentAnalysisTool()
scrape_tool = ScrapeWebsiteTool()
instructions_path = "data/instructions"
directory_read_tool = DirectoryReadTool(directory=instructions_path)

# Planner Crew

### Agents

In [ ]:
planner = Agent(
    role=research_data['agents'][0]['role'],
    goal=research_data['agents'][0]['goal'],
    backstory=research_data['agents'][0]['backstory'],
    allow_delegation=False,
    verbose=True, 
    llm=llm
)
writer = Agent(
    role=research_data['agents'][1]['role'],
    goal=research_data['agents'][1]['goal'],
    backstory=research_data['agents'][1]['backstory'],
    allow_delegation=False,
    verbose=True, 
    llm=llm
)
editor = Agent(
    role=research_data['agents'][2]['role'],
    goal=research_data['agents'][2]['goal'],
    backstory=research_data['agents'][2]['backstory'],
    allow_delegation=False,
    verbose=True, 
    llm=llm
)

### Tasks

In [ ]:
plan = Task(
    description=research_data['tasks'][0]['description'],
    expected_output=research_data['tasks'][0]['expected_output'],
    agent=planner
)
write = Task(
    description=research_data['tasks'][1]['description'],
    expected_output=research_data['tasks'][1]['expected_output'],
    agent=writer
)
edit= Task(
    description=research_data['tasks'][2]['description'],
    expected_output=research_data['tasks'][2]['expected_output'],
    agent=editor
)

### CREW

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=0
)

In [ ]:
result = crew.kickoff(
    inputs={"topic": "Artificial Intelligents"}
)

In [ ]:
print(f"Model: {model}")
Markdown(result)

# Customer Support System

In [ ]:
support_agent = Agent(
    role=customer_support_data['agents'][0]['role'],
    goal=customer_support_data['agents'][0]['goal'],
    backstory=customer_support_data['agents'][0]['backstory'],
    allow_delegation=False,
    verbose=True,
    llm=llm
)
support_quality_agent = Agent(
    role=customer_support_data['agents'][1]['role'],
    goal=customer_support_data['agents'][1]['goal'],
    backstory=customer_support_data['agents'][1]['backstory'],
    verbose=True,
    llm=llm
)


In [ ]:
inquiry_task = Task(
    description=customer_support_data['tasks'][0]['description'],
    expected_output=customer_support_data['tasks'][0]['expected_output'],
    tools=[scrape_tool],
    agent=support_agent
)
quality_assurance_review = Task(
    description=customer_support_data['tasks'][1]['description'],
    expected_output=customer_support_data['tasks'][1]['expected_output'],
    agent=support_quality_agent
)

support_crew = Crew(
    agents=[support_agent, support_quality_agent],
    tasks=[inquiry_task, quality_assurance_review],
    verbose=2, 
    memory=True
)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
support_result = support_crew.kickoff(inputs=inputs)

In [ ]:
Markdown(support_result)

# VARIABLES UPDATES

In [ ]:
model = "gpt-4o"
llm = ChatOpenAI(openai_api_key=api_key, model=model)

# Sales Crew

In [ ]:

sales_rep = Agent(
    role=sales_data['agents'][0]['role'],
    goal=sales_data['agents'][0]['goal'],
    backstory=sales_data['agents'][0]['backstory'],
    allow_delegation=False,
    verbose=True,
    llm=llm
)

lead_sales_rep = Agent(
    role=sales_data['agents'][1]['role'],
    goal=sales_data['agents'][1]['goal'],
    backstory=sales_data['agents'][1]['backstory'],
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
profiling_task = Task(
    description=sales_data['tasks'][0]['description'],
    expected_output=sales_data['tasks'][0]['expected_output'],
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep
)

outrich_task = Task(
    description=sales_data['tasks'][1]['description'],
    expected_output=sales_data['tasks'][1]['expected_output'],
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep
)

sales_crew = Crew(
    agents=[sales_rep, lead_sales_rep],
    tasks=[profiling_task, outrich_task],
    verbose=2,
	memory=True
)

inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

sales_result = sales_crew.kickoff(inputs=inputs)


In [ ]:
Markdown(sales_result)

# Event Crew

In [ ]:
venue_coordinator = Agent(
    role=events_data['agents'][0]['role'],
    goal=events_data['agents'][0]['goal'],
    backstory=events_data['agents'][0]['backstory'],
    verbose=True,
    llm=llm,
    tools=[search_tool, scrape_tool]
)

logistics_coordinator = Agent(
    role=events_data['agents'][1]['role'],
    goal=events_data['agents'][1]['goal'],
    backstory=events_data['agents'][1]['backstory'],
    verbose=True,
    llm=llm,
    tools=[search_tool, scrape_tool]
)

marketing_coordinator = Agent(
    role=events_data['agents'][2]['role'],
    goal=events_data['agents'][2]['goal'],
    backstory=events_data['agents'][2]['backstory'],
    verbose=True,
    llm=llm,
    tools=[search_tool, scrape_tool]
)

In [ ]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [ ]:
venue_task = Task(
    description=events_data['tasks'][0]['description'],
    expected_output=events_data['tasks'][0]['expected_output'],
    agent=venue_coordinator,
    human_input=True, 
    output_json=VenueDetails,
    output_file=venue_json_path
)

logistics_task = Task(
    description=events_data['tasks'][1]['description'],
    expected_output=events_data['tasks'][1]['expected_output'],
    agent=logistics_coordinator,
    human_input=True,
    async_execution=True
)
marketing_task = Task(
    description=events_data['tasks'][2]['description'],
    expected_output=events_data['tasks'][2]['expected_output'],
    agent=marketing_coordinator,
    async_execution=True, 
    output_file="marketing_report.md"
)

event_crew = Crew(
    agents=[venue_coordinator, logistics_coordinator, marketing_coordinator],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=2
)

event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [ ]:
event_results = event_crew.kickoff(inputs=event_details)

In [ ]:
print_json(venue_json_path)

In [ ]:
Markdown("marketing_report.md")

# Financial Crew

In [ ]:
data_analyst_agent = Agent(
    role=financial_data['agents'][0]['role'],
    goal=financial_data['agents'][0]['goal'],
    backstory=financial_data['agents'][0]['backstory'],
    verbose=True,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    llm=llm
)

trading_strategy_agent = Agent(
    role=financial_data['agents'][1]['role'],
    goal=financial_data['agents'][1]['goal'],
    backstory=financial_data['agents'][1]['backstory'],
    verbose=True,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    llm=llm
)

execution_agent = Agent(
    role=financial_data['agents'][2]['role'],
    goal=financial_data['agents'][2]['goal'],
    backstory=financial_data['agents'][2]['backstory'],
    verbose=True,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    llm=llm
)

risk_management_agent = Agent(
    role=financial_data['agents'][3]['role'],
    goal=financial_data['agents'][3]['goal'],
    backstory=financial_data['agents'][3]['backstory'],
    verbose=True,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    llm=llm
)
    

In [ ]:
data_task = Task(
    description=financial_data['tasks'][0]['description'],
    expected_output=financial_data['tasks'][0]['expected_output'],
    agent=data_analyst_agent
)

strategy_task = Task(
    description=financial_data['tasks'][1]['description'],
    expected_output=financial_data['tasks'][1]['expected_output'],
    agent=trading_strategy_agent
)

execution_task = Task(
    description=financial_data['tasks'][2]['description'],
    expected_output=financial_data['tasks'][2]['expected_output'],
    agent=execution_agent
)

risk_task = Task(
    description=financial_data['tasks'][3]['description'],
    expected_output=financial_data['tasks'][3]['expected_output'],
    agent=risk_management_agent
)

In [ ]:
financial_crew = Crew(
    agents=[data_analyst_agent, trading_strategy_agent, execution_agent, risk_management_agent],
    tasks=[data_task, strategy_task, execution_task, risk_task],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(openai_api_key=api_key, model=model, temperature=0.7),
    verbose=True
)

financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
financial_result = financial_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
Markdown(financial_result)

# Resume Builder Crew

In [ ]:
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=resume_data_path)
semantic_search_resume = MDXSearchTool(mdx=resume_data_path)
output_path_resume = "output/tailored_resume.md"
output_path_interview_prep = "output/interview_prep.md"


In [ ]:
researcher_agent = Agent(
    role=resume_data['agents'][0]['role'],
    goal=resume_data['agents'][0]['goal'],
    backstory=resume_data['agents'][0]['backstory'],
    verbose=True,
    llm=llm,
    tools=[scrape_tool, search_tool]
)

profiler_agent = Agent(
    role=resume_data['agents'][1]['role'],
    goal=resume_data['agents'][1]['goal'],
    backstory=resume_data['agents'][1]['backstory'],
    verbose=True,
    llm=llm,
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume]
)

resulme_strategist_agent = Agent(
    role=resume_data['agents'][2]['role'],
    goal=resume_data['agents'][2]['goal'],
    backstory=resume_data['agents'][2]['backstory'],
    verbose=True,
    llm=llm,
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume]
)

interview_preparation_agent = Agent(
    role=resume_data['agents'][3]['role'],
    goal=resume_data['agents'][3]['goal'],
    backstory=resume_data['agents'][3]['backstory'],
    verbose=True,
    llm=llm,
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume]
)

In [ ]:
research_task = Task(
    description=resume_data['tasks'][0]['description'],
    expected_output=resume_data['tasks'][0]['expected_output'],
    agent=researcher_agent, 
    async_execution=True
)
profile_task = Task(
    description=resume_data['tasks'][1]['description'],
    expected_output=resume_data['tasks'][1]['expected_output'],
    agent=profiler_agent,
    async_execution=True
)
resume_strategy_task = Task(
    description=resume_data['tasks'][2]['description'],
    expected_output=resume_data['tasks'][2]['expected_output'],
    agent=resulme_strategist_agent,
    output_file=output_path_resume,
    context=[research_task, profile_task],
)

interview_prep_task = Task(
    description=resume_data['tasks'][3]['description'],
    expected_output=resume_data['tasks'][3]['expected_output'],
    agent=interview_preparation_agent,
    output_file=output_path_interview_prep,
    context=[research_task, profile_task, resume_strategy_task],
)

In [ ]:
job_application_crew = Crew(
    agents=[researcher_agent, profiler_agent, resulme_strategist_agent, interview_preparation_agent],
    tasks=[research_task, profile_task, resume_strategy_task, interview_prep_task],
    verbose=2
)

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
job_application_result = job_application_crew.kickoff(inputs=job_application_inputs)